In this notebook, I will experiment with a simple backtesting strategy using the `backtesting.py` library. The strategy will be tested on Bitcoin/USD 1-hour interval data. This workflow will demonstrate how to set up, implement, and evaluate a trading strategy using historical data.


## 01_backtesting-py_simple.ipynb
- **Goal:** Evaluate a long-only SMA crossover strategy on BTCUSDT hourly data with the `backtesting.py` framework.
- **Data preparation:** Retrieves candles through `src.data.data_pipeline.get_historical_data`, then converts the Polars DataFrame to pandas for compatibility with `Backtest`.
- **Strategy logic:** Implements `SmaCross_bt`, which tracks short/long simple moving averages, buys on upward crossovers, and exits on downward crossovers.
- **Backtest run:** Executes `Backtest` with $1M starting cash, 0.2% commission, and exclusive orders; prints the resulting performance statistics and generates the HTML equity plot.
- **Optimization experiments:** Runs a 300-trial Optuna study optimizing both return and Sharpe ratio (multi-objective) followed by a single-objective variant that combines the same metrics, then re-runs the backtest with the chosen parameters (`n_short=47`, `n_long=67`).

In [1]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [3]:
from src.data.data_pipeline import get_historical_data
from src.utils.utils import polars_to_pandas

In [1]:
from emrpy import get_root_path
get_root_path()

PosixPath('/Users/ezequielmrivero/git_tradelab/Crypto-Backtester-Duel/notebooks')

In [4]:
df = get_historical_data(download=False)
df.head()

FileNotFoundError: No such file or directory (os error 2): ...rivero/git_tradelab/Crypto-Backtester-Duel/notebooks/data/BTCUSDT_1h_tmp_365days.parquet (set POLARS_VERBOSE=1 to see full path)

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


## Defining strategy

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from backtesting import Backtest

class SmaCross_bt(Strategy):
    """
    A simple SMA Cross strategy:
    - If short SMA crosses above long SMA, enter a long position.
    - If short SMA crosses below long SMA, close the position.
    """

    n_short = 30
    n_long = 100

    def init(self):
        # Convert the series to indicators used by backtesting.py
        price = self.data.Close
        self.sma_short = self.I(SMA, price, self.n_short)
        self.sma_long = self.I(SMA, price, self.n_long)

    def next(self):
        # If short SMA crosses above long SMA, and not already in a trade:
        if crossover(self.sma_short, self.sma_long):
            self.buy()

        # If short SMA crosses below long SMA, close any open position:
        elif crossover(self.sma_long, self.sma_short):
            self.position.close()

/Users/ezequielmrivero/git_projects/algo_repos/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ezequielmrivero/git_projects/algo_repos/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
def run_bt_sma_backtest(df_pl) -> None:
    df_pd = polars_to_pandas(df_pl)
    bt = Backtest(df_pd, SmaCross_bt, cash=1_000_000, commission=0.002, exclusive_orders=True)
    stats = bt.run()
    return stats, bt

stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-04-18 05:00:00
End                       2025-04-18 04:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    51.01598
Equity Final [$]                1091072.54514
Equity Peak [$]                  1596227.5031
Commissions [$]                  227316.29478
Return [%]                            9.10725
Buy & Hold Return [%]                28.35305
Return (Ann.) [%]                     9.08127
Volatility (Ann.) [%]                39.58861
CAGR [%]                              9.10834
Sharpe Ratio                          0.22939
Sortino Ratio                          0.4289
Calmar Ratio                          0.25863
Alpha [%]                            -3.20992
Beta                                  0.43442
Max. Drawdown [%]                   -35.11348
Avg. Drawdown [%]                    -2.46728
Max. Drawdown Duration      150 days 17:00:00
Avg. Drawdown Duration        6 days 23:00:00
# Trades                          

GridPlot(id='p1347', ...)

In [ ]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def multi_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    # Optuna multi-objective: return a tuple
    return ret, sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    directions=['maximize', 'maximize'],
    study_name='sma_cross_multiobj'
)

# Run optimization
study.optimize(multi_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: Return={t.values[0]:.2f}%, Sharpe={t.values[1]:.2f}, "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-05-19 22:38:12,236] A new study created in memory with name: sma_cross_multiobj
[I 2025-05-19 22:38:12,283] Trial 0 finished with values: [-10.662569107999884, -0.3382185279019402] and parameters: {'n_short': 42, 'n_long': 165}.
[I 2025-05-19 22:38:12,328] Trial 1 finished with values: [-1.810060571999976, -0.051056933918418666] and parameters: {'n_short': 25, 'n_long': 125}.
[I 2025-05-19 22:38:12,370] Trial 2 finished with values: [-6.749926463999994, -0.21676786842197976] and parameters: {'n_short': 47, 'n_long': 192}.
[I 2025-05-19 22:38:12,410] Trial 3 finished with values: [-13.347406279999937, -0.44446598092997436] and parameters: {'n_short': 44, 'n_long': 182}.
[I 2025-05-19 22:38:12,453] Trial 4 finished with values: [3.43499688200003, 0.09126844648626507] and parameters: {'n_short': 45, 'n_long': 50}.
[I 2025-05-19 22:38:12,493] Trial 5 finished with values: [-12.931060562000027, -0.42856786208856074] and parameters: {'n_short': 41, 'n_long': 183}.
[I 2025-05-19 22:38

Number of Pareto-optimal trials: 1
  trial#201: Return=47.62%, Sharpe=0.84, n_short=48, n_long=71


In [ ]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def single_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    return ret + 10*sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    direction='maximize',
    study_name='sma_cross_singleobj'
)

# Run optimization
study.optimize(single_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-05-19 22:38:25,605] A new study created in memory with name: sma_cross_singleobj
[I 2025-05-19 22:38:25,648] Trial 0 finished with value: -29.04802626625433 and parameters: {'n_short': 15, 'n_long': 198}. Best is trial 0 with value: -29.04802626625433.
[I 2025-05-19 22:38:25,691] Trial 1 finished with value: -28.669993668888026 and parameters: {'n_short': 11, 'n_long': 85}. Best is trial 1 with value: -28.669993668888026.
[I 2025-05-19 22:38:25,734] Trial 2 finished with value: -13.03467824020944 and parameters: {'n_short': 6, 'n_long': 151}. Best is trial 2 with value: -13.03467824020944.
[I 2025-05-19 22:38:25,775] Trial 3 finished with value: -34.311486982656824 and parameters: {'n_short': 7, 'n_long': 90}. Best is trial 2 with value: -13.03467824020944.
[I 2025-05-19 22:38:25,819] Trial 4 finished with value: 18.773614282629584 and parameters: {'n_short': 30, 'n_long': 116}. Best is trial 4 with value: 18.773614282629584.
[I 2025-05-19 22:38:25,863] Trial 5 finished with va

Number of Pareto-optimal trials: 4
  trial#208: n_short=46, n_long=67
  trial#265: n_short=46, n_long=67
  trial#289: n_short=46, n_long=67
  trial#294: n_short=46, n_long=67


In [ ]:
SmaCross_bt.n_short = 47
SmaCross_bt.n_long  = 67
stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-04-18 05:00:00
End                       2025-04-18 04:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    50.99315
Equity Final [$]                1546408.47854
Equity Peak [$]                 1977205.05482
Commissions [$]                  406159.29146
Return [%]                           54.64085
Buy & Hold Return [%]                30.37102
Return (Ann.) [%]                    54.45677
Volatility (Ann.) [%]                58.25963
CAGR [%]                             54.64854
Sharpe Ratio                          0.93473
Sortino Ratio                         2.69825
Calmar Ratio                          1.93195
Alpha [%]                            40.74698
Beta                                  0.45747
Max. Drawdown [%]                   -28.18741
Avg. Drawdown [%]                    -1.75589
Max. Drawdown Duration      121 days 14:00:00
Avg. Drawdown Duration        4 days 01:00:00
# Trades                          

GridPlot(id='p1723', ...)